## Turing Machine Simulator

_csc427, semester 232_
<br>
_university of miami_
<br>
_date: 5 march 2023_



### Syntax 


The file is a sequence of stanzas. Each stanza starts with a head line starting 
in the first column. The first word in the head line as a keyword indicating 
the type of stanza. A stanza may have continuation lines, each are indented.

The # character starts a comment, and the comment extends to the end of the line. 
Since the # is used for this 
purpose it cannot be a tape symbol. If you wish to code programs in the textbook that use
the # symbol, substitute with one of the allowed punctuations.

The syntax can be described by the following grammar.

- __M__ -> (__Stanza__ [emptyline])*
- __Stanza__ -> __StartStanza__ | __TapesStanza__ | __AcceptStanza__ | __RejectStanza__ | __StateStanza__
- __StartStanza__ -> "start:" __Ident__
- __TapesStanza__ -> "tapes:" number
- __AcceptStanza__ -> "accept:" __Ident__ (\n\t __Ident__)*
- __RejectStanza__ -> "reject:" __Ident__ (\n\t __Ident__)*
- __StateStanza__ -> "state:" __Ident__ (\n\t __StateTransition__)+
- __StateTransition__ -> (__Symbol__|__Special__){k} (__Symbol__|__Special__){k} __Action__ __Ident__
- __Symbol__ -> tape symbols are alphanumeric or punctuation ! $ % & ( ) * + , - . or /
- __Special__ -> the : and _
- __Action__ -> the characters l, r and n or uppercase L, R and N.
- __Ident__ -> a nonempty string of alphanumerics

There must be exactly one start, accept and reject stanzas.

The tape stanza must occur before any state stanza.

The underscore (_) substitutes for the blank. It is the default chacters of all unfilled cells on the tape.
Using the blank for the initial tape contents is allowed, it will be rewritten to the underscore character.

An Ident is the label of a state.

A StateStanza names to from state after the colon and each StateTransition line gives
one transition according to the syntax (for the case of k=1):

<pre>    read-symbol write-symbol action new-state </pre>

For the case of k&gt;1, 
The StateTransition line is interpreted as first the k tape symbols to match on the k tapes; then the k tape symbols to write on the k tapes, then the k actions to undertake on the k tapes.

The action is either l, r or n, meaning move left, move right, or no move. As a debuging feature, if the code for the action captialized the machine configuration is printed after the transition.

The colon (:) is a wildcard. It's use among the k read symbols matches any symbol. Four use cases are allowed, and are listed in the priority the case is applied,

    No wildcards. An exact match of the k type symbols has top precedence.
    One wildcard. An exact match for all but one symbol is tried if 
       there is no exact match.
    k-1 wildcards. An exact match for exactly one symbol, the k-1 are wildcards, 
       is tried next.
    All wildcards. The default matches anything.

When the wildcard appearing as a write symbol, it is set equal to the read symbol, on the corresponding tape.

A missing transition halts with reject. This and the wildcard are convenience features to
shorten the TM programs.

If the machine rejects, it can be queried for the cause of the reject,

    Reject becaues halted in a reject state.
    Reject because of a missing transition.
    Reject bedcause the computation terminated for excessive computation steps.

The method is_exception classes the first two rejects as correct computations, and the last as an exception.

### Recognizing a Simple RE

An example for the syntax is the following program recognizing a+b+c+. 

Note the call to compute_tm requires a string, a parameter giving maximum number of steps
permitted to the computation, and an optional verbose parameter.

In [11]:
tm_abc= """# a TM program to recognize a+b+c+

accept: A
reject: R
start: q0

state: q0    
    a a r q0  # should start with an a, and loop over further a's
    b b r q1  # until a b.
              # note: missing transtions cause the machine to reject (c or _)
    
state: q1
    b b r q1  # continue over further b's
    c c r q2  # until a c.
    
state: q2
    c c r q2  # continue over further c's
    _ _ r A   # until the end of the tape

"""

# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100

from turing_machine_sim import *

tm = MachineParser.create_from_description(tm_abc)
tm.compute_tm('aabbcc', max_steps, verbose=verbose)
tm.compute_tm('abca', max_steps, verbose=verbose)



0 [q0]	[a]abbcc_
1 [q0]	a[a]bbcc_
2 [q0]	aa[b]bcc_
3 [q1]	aab[b]cc_
4 [q1]	aabb[c]c_
5 [q2]	aabbc[c]_
6 [q2]	aabbcc[_]
7 [A]	aabbcc_[_]
accept (ok)
0 [q0]	[a]bca_
1 [q0]	a[b]ca_
2 [q1]	ab[c]a_
3 [q2]	abc[a]_
reject (transition missing)


False

### Erasing the Tape

Accept everything, while erasing the tape leaving the head at the start of the tape.

A Turing Machine opens up another aspect of computation. So far, the algorithms in this
class have only a boolean result: true or false for membership of an element in a set.
(In fact, membership in a subset of a set, as we always assume the input is of proper form.)

This example intrduces another meaning for computation. It is the more common meaning of
"doing something" &mdash; in this case, the TM erases the tape.

In [12]:
tm_erase = """# erase the tape

accept: A
reject: R
start: q0

state: q0    # write a tape endmarker
    : $ R q1
    
state: q1
    : : r q1 # more right not replacing symbols
    _ _ l q2 # begin the erase while returning
    
state: q2
    : _ l q2 # erase symbols
    $ _ N A  # stop at the endmarker
"""


# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100

tm = MachineParser.create_from_description(tm_erase)
tm.compute_tm('abc', max_steps, verbose=verbose)


0 [q0]	[a]bc_
1 [q1]	$[b]c_
2 [q1]	$b[c]_
3 [q1]	$bc[_]
4 [q2]	$b[c]_
5 [q2]	$[b]__
6 [q2]	[$]___
7 [A]	[_]___
accept (ok)


True